In [1]:
import sys, os, gc
import subprocess
from math import *
import numpy as np
from datetime import *
from pylab import *
import random
from datetime import datetime
import json
import imgaug as ia
import imgaug.augmenters as iaa
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.utils import to_categorical

In [2]:
def vgg6(input_shape=(128, 128, 3)):


    model = tf.keras.models.Sequential(name='VGG6')

    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=input_shape, name='conv1'))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', name='conv2'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', name='conv3'))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', name='conv4'))
    model.add(tf.keras.layers.BatchNormalization(axis = 3, name = 'bn_2'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(4, 4)))
    model.add(tf.keras.layers.Dropout(0.25))
    
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', name='conv5'))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', name='conv6'))
    model.add(tf.keras.layers.BatchNormalization(axis = 3, name = 'bn_3'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(4, 4)))

    ## Fully connected layers
    model.add(tf.keras.layers.Flatten())
    
    model.add(tf.keras.layers.Dense(256, activation='relu', name='fc_2'))

       
    model.add(tf.keras.layers.Dense(128, activation='relu', name='fc_3'))
    
    model.add(tf.keras.layers.Dense(64, activation='relu', name='fc_4'))

    model.add(tf.keras.layers.Dense(8, activation='relu', name='fc_5'))
    
    # output layer
    model.add(tf.keras.layers.Dense(1, activation='tanh', name='fc_out2'))

    return model

def esn_shuffle(array, seed=0):
        np.random.seed(seed)
        np.random.shuffle(array)
        return array

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)


In [3]:
model = vgg6()
model.summary()

Model: "VGG6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 126, 126, 128)     3584      
_________________________________________________________________
conv2 (Conv2D)               (None, 124, 124, 128)     147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 62, 62, 128)       0         
_________________________________________________________________
conv3 (Conv2D)               (None, 60, 60, 256)       295168    
_________________________________________________________________
conv4 (Conv2D)               (None, 58, 58, 256)       590080    
_________________________________________________________________
bn_2 (BatchNormalization)    (None, 58, 58, 256)       1024   

In [4]:
def esn_shuffle(array, seed=None):
        if seed is not None:
            np.random.seed(seed)
        np.random.shuffle(array)
        return array

In [5]:
seq = iaa.Sequential([
    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.1))),
    iaa.GammaContrast(gamma=(0.97,1.03)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 3), per_channel=0.5),
    iaa.Add((-5, 5), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.5),
    iaa.Sometimes(0.1, iaa.Grayscale(alpha=(0.0, 1.0))),
    iaa.Sometimes(0.50, iaa.Grayscale(alpha=1.)),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Affine(
        rotate=(0, 359),
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, 
        mode=ia.ALL, cval=(0, 255))
    ], random_order=True)


seqGray = iaa.Sequential([
    iaa.GammaContrast(gamma=(0.97,1.03)),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Affine(
        rotate=(0, 359),
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, 
        mode=ia.ALL, cval=(255, 255))
    ], random_order=True)

In [6]:
%%time 

data = np.load('samples/RGB_128x128_test_000.npz')
images_RGB = data['images'].astype(np.uint8)
labels_RGB = data['labels'] 
pgcIDs_RGB = data['pgcIDs']
N = images_RGB.shape[0]
filter_RGB = np.chararray(N)
filter_RGB[:] = 'c'
data.close() 
del data


data = np.load('samples/g_128x128_test_000.npz')
images_g = data['images'].astype(np.uint8)
labels_g = data['labels'] 
pgcIDs_g = data['pgcIDs']
N = images_g.shape[0]
filter_g = np.chararray(N)
filter_g[:] = 'g'
data.close() 
del data

data = np.load('samples/r_128x128_test_000.npz')
images_r = data['images'].astype(np.uint8)
labels_r = data['labels'] 
pgcIDs_r = data['pgcIDs']
N = images_r.shape[0]
filter_r = np.chararray(N)
filter_r[:] = 'r'
data.close() 
del data

data = np.load('samples/i_128x128_test_000.npz')
images_i = data['images'].astype(np.uint8)
labels_i = data['labels'] 
pgcIDs_i = data['pgcIDs']
N = images_i.shape[0]
filter_i = np.chararray(N)
filter_i[:] = 'i'
data.close() 
del data

CPU times: user 2.36 s, sys: 96.3 ms, total: 2.46 s
Wall time: 2.46 s


In [7]:
images_gri = np.concatenate((images_g, images_r, images_i))
labels_gri = np.concatenate((labels_g, labels_r, labels_i))
pgcIDs_gri = np.concatenate((pgcIDs_g, pgcIDs_r, pgcIDs_i))
filter_gri = np.concatenate((filter_g, filter_r, filter_i))

N_RGB = len(labels_RGB)
N_gri = len(labels_gri)

In [8]:
ia.seed(100)

indx = esn_shuffle(np.arange(N_gri), seed=200)
images_aug = seqGray(images=images_gri[indx][:N_RGB,:,:,:])
labels_aug = labels_gri[indx][:N_RGB]
pgcIDs_aug = pgcIDs_gri[indx][:N_RGB]
filter_aug = filter_gri[indx][:N_RGB]

n = len(images_aug)
p = int(n/2)
images_aug[:p] = 255 - images_aug[:p]

ia.seed(200)

images_aug = np.concatenate((seq(images=images_RGB),images_aug))
labels_aug = np.concatenate((labels_RGB,labels_aug))
pgcIDs_aug = np.concatenate((pgcIDs_RGB,pgcIDs_aug))
filter_aug = np.concatenate((filter_RGB,filter_aug))

indx = np.arange(len(images_aug))
indx = esn_shuffle(indx, seed=100)
images_test_aug = images_aug[indx]
labels_test_aug = labels_aug[indx]
pgcIDs_test_aug = pgcIDs_aug[indx]
filter_test_aug = filter_aug[indx]

labels_test_aug = 2.*(labels_test_aug-45.)/45. - 1.

In [9]:
batches = esn_shuffle(np.arange(50), seed=50)

for i in range(1,60):
    batches = np.concatenate((batches, esn_shuffle(np.arange(50), seed=i*50))) 

batches, len(batches)

(array([34, 36,  1, ...,  2, 29,  7]), 3000)

In [10]:
zp_dir = 'Uset2_npz/'
ckpt_dir = 'U2_model05_ckpt/'

def trainer(iter, batches, suffix=None):

  if suffix is None:
    suffix = ''

  batchNo = batches[iter]
  if True: 

    vgg_model = vgg6()
    vgg_model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(), metrics=['mse', 'mae'])

    if iter>0:

      vgg_model = vgg6()
      vgg_model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(), metrics=['mse', 'mae'])
      vgg_model.load_weights(ckpt_dir+str(iter-1)+suffix+".ckpt")

      # if batchNo % 2 == 0 : 
      #   for layer in vgg_model.layers[:-1]:
      #     layer.trainable = False

      with open(ckpt_dir+'evalDict'+suffix+'.json') as json_file:
        evalDict = json.load(json_file)
    else:
      evalDict = {}


    print(iter, batchNo)


    batchFile = npzname = '128x128_train_aug_'+'%02d'%(batchNo+1)+'.npz'
    data = np.load(zp_dir + batchFile)
    images_train_aug = data['images'].astype(np.uint8)
    labels_train_aug = data['labels']
    pgcIDs_train_aug = data['pgcIDs']
    data.close() 
    del data

    labels_train_aug = 2.*(labels_train_aug-45.)/45. - 1.
    
    n_epochs=1
    vgg_model.fit(images_train_aug, labels_train_aug, 
                                      epochs=n_epochs, batch_size=64, 
                                      validation_data=(images_test_aug, labels_test_aug),
                                      verbose=1, shuffle=True)
    evalDict[iter] = {} 
    for key in vgg_model.history.history:
      evalDict[iter][key] = vgg_model.history.history[key][0]
    evalDict[iter]["batchNo"] = batchNo
    evalDict[iter]["batchFile"] = batchFile
    
    with open(ckpt_dir+"evalDict"+suffix+".json", "w", encoding ='utf8') as outfile:
      json.dump(evalDict, outfile, allow_nan=True, cls=NpEncoder)
    
    vgg_model.save_weights(ckpt_dir+str(iter)+suffix+".ckpt")

    del vgg_model
    tf.keras.backend.clear_session()
    del images_train_aug 
    del labels_train_aug
    del pgcIDs_train_aug

    gc.collect()

    return evalDict

In [11]:
with tf.device('/gpu:1'):
    for i in range(0, 1000):
        trainer(i, batches)

0 34
282/282 [==============================] - 192s 680ms/step - loss: 0.2306 - mse: 0.2306 - mae: 0.3893 - val_loss: 0.2491 - val_mse: 0.2491 - val_mae: 0.4107
1 36
282/282 [==============================] - 77s 274ms/step - loss: 0.1737 - mse: 0.1737 - mae: 0.3294 - val_loss: 0.1572 - val_mse: 0.1572 - val_mae: 0.3060
2 1
282/282 [==============================] - 73s 260ms/step - loss: 0.1505 - mse: 0.1505 - mae: 0.3012 - val_loss: 0.1069 - val_mse: 0.1069 - val_mae: 0.2478
3 38
282/282 [==============================] - 73s 260ms/step - loss: 0.1210 - mse: 0.1210 - mae: 0.2677 - val_loss: 0.1023 - val_mse: 0.1023 - val_mae: 0.2471
4 8
282/282 [==============================] - 73s 260ms/step - loss: 0.1099 - mse: 0.1099 - mae: 0.2543 - val_loss: 0.1132 - val_mse: 0.1132 - val_mae: 0.2590
5 16
282/282 [==============================] - 73s 259ms/step - loss: 0.1009 - mse: 0.1009 - mae: 0.2434 - val_loss: 0.1144 - val_mse: 0.1144 - val_mae: 0.2495
6 35
282/282 [=====================

51 36
282/282 [==============================] - 73s 259ms/step - loss: 0.0382 - mse: 0.0382 - mae: 0.1427 - val_loss: 0.0455 - val_mse: 0.0455 - val_mae: 0.1547
52 1
282/282 [==============================] - 73s 259ms/step - loss: 0.0349 - mse: 0.0349 - mae: 0.1361 - val_loss: 0.0451 - val_mse: 0.0451 - val_mae: 0.1547
53 38
282/282 [==============================] - 73s 259ms/step - loss: 0.0337 - mse: 0.0337 - mae: 0.1349 - val_loss: 0.0401 - val_mse: 0.0401 - val_mae: 0.1460
54 8
282/282 [==============================] - 73s 259ms/step - loss: 0.0363 - mse: 0.0363 - mae: 0.1390 - val_loss: 0.0303 - val_mse: 0.0303 - val_mae: 0.1260
55 16
282/282 [==============================] - 73s 259ms/step - loss: 0.0343 - mse: 0.0343 - mae: 0.1358 - val_loss: 0.0450 - val_mse: 0.0450 - val_mae: 0.1536
56 35
282/282 [==============================] - 73s 259ms/step - loss: 0.0350 - mse: 0.0350 - mae: 0.1369 - val_loss: 0.0469 - val_mse: 0.0469 - val_mae: 0.1553
57 13
282/282 [===============

102 37
282/282 [==============================] - 73s 259ms/step - loss: 0.0267 - mse: 0.0267 - mae: 0.1198 - val_loss: 0.0393 - val_mse: 0.0393 - val_mae: 0.1434
103 28
282/282 [==============================] - 73s 259ms/step - loss: 0.0274 - mse: 0.0274 - mae: 0.1210 - val_loss: 0.0289 - val_mse: 0.0289 - val_mae: 0.1209
104 43
282/282 [==============================] - 73s 259ms/step - loss: 0.0269 - mse: 0.0269 - mae: 0.1206 - val_loss: 0.0290 - val_mse: 0.0290 - val_mae: 0.1235
105 49
282/282 [==============================] - 73s 259ms/step - loss: 0.0270 - mse: 0.0270 - mae: 0.1206 - val_loss: 0.0282 - val_mse: 0.0282 - val_mae: 0.1191
106 5
282/282 [==============================] - 73s 259ms/step - loss: 0.0267 - mse: 0.0267 - mae: 0.1196 - val_loss: 0.0289 - val_mse: 0.0289 - val_mae: 0.1219
107 33
282/282 [==============================] - 73s 259ms/step - loss: 0.0278 - mse: 0.0278 - mae: 0.1215 - val_loss: 0.0348 - val_mse: 0.0348 - val_mae: 0.1341
108 20
282/282 [=======

282/282 [==============================] - 73s 259ms/step - loss: 0.0218 - mse: 0.0218 - mae: 0.1084 - val_loss: 0.0281 - val_mse: 0.0281 - val_mae: 0.1213
203 28
282/282 [==============================] - 73s 259ms/step - loss: 0.0219 - mse: 0.0219 - mae: 0.1089 - val_loss: 0.0247 - val_mse: 0.0247 - val_mae: 0.1105
204 20
282/282 [==============================] - 73s 259ms/step - loss: 0.0214 - mse: 0.0214 - mae: 0.1077 - val_loss: 0.0265 - val_mse: 0.0265 - val_mae: 0.1125
205 49
282/282 [==============================] - 73s 259ms/step - loss: 0.0210 - mse: 0.0210 - mae: 0.1062 - val_loss: 0.0265 - val_mse: 0.0265 - val_mae: 0.1146
206 18
282/282 [==============================] - 73s 259ms/step - loss: 0.0225 - mse: 0.0225 - mae: 0.1099 - val_loss: 0.0282 - val_mse: 0.0282 - val_mae: 0.1182
207 17
282/282 [==============================] - 73s 259ms/step - loss: 0.0224 - mse: 0.0224 - mae: 0.1090 - val_loss: 0.0257 - val_mse: 0.0257 - val_mae: 0.1123
208 44
282/282 [=============

282/282 [==============================] - 73s 258ms/step - loss: 0.0175 - mse: 0.0175 - mae: 0.0980 - val_loss: 0.0237 - val_mse: 0.0237 - val_mae: 0.1071
303 36
282/282 [==============================] - 73s 258ms/step - loss: 0.0191 - mse: 0.0191 - mae: 0.1013 - val_loss: 0.0257 - val_mse: 0.0257 - val_mae: 0.1128
304 24
282/282 [==============================] - 73s 258ms/step - loss: 0.0188 - mse: 0.0188 - mae: 0.1008 - val_loss: 0.0241 - val_mse: 0.0241 - val_mae: 0.1094
305 8
282/282 [==============================] - 73s 258ms/step - loss: 0.0189 - mse: 0.0189 - mae: 0.1014 - val_loss: 0.0320 - val_mse: 0.0320 - val_mae: 0.1254
306 28
282/282 [==============================] - 73s 258ms/step - loss: 0.0184 - mse: 0.0184 - mae: 0.1002 - val_loss: 0.0234 - val_mse: 0.0234 - val_mae: 0.1074
307 25
282/282 [==============================] - 73s 258ms/step - loss: 0.0184 - mse: 0.0184 - mae: 0.1006 - val_loss: 0.0226 - val_mse: 0.0226 - val_mae: 0.1054
308 6
282/282 [===============

...

282/282 [==============================] - 73s 258ms/step - loss: 0.0148 - mse: 0.0148 - mae: 0.0905 - val_loss: 0.0261 - val_mse: 0.0261 - val_mae: 0.1098
503 22
282/282 [==============================] - 73s 258ms/step - loss: 0.0151 - mse: 0.0151 - mae: 0.0916 - val_loss: 0.0264 - val_mse: 0.0264 - val_mae: 0.1117
504 45
282/282 [==============================] - 73s 258ms/step - loss: 0.0158 - mse: 0.0158 - mae: 0.0930 - val_loss: 0.0257 - val_mse: 0.0257 - val_mae: 0.1086
505 44
282/282 [==============================] - 73s 258ms/step - loss: 0.0154 - mse: 0.0154 - mae: 0.0920 - val_loss: 0.0270 - val_mse: 0.0270 - val_mae: 0.1119
506 40
282/282 [==============================] - 73s 258ms/step - loss: 0.0161 - mse: 0.0161 - mae: 0.0938 - val_loss: 0.0244 - val_mse: 0.0244 - val_mae: 0.1070
507 12
282/282 [==============================] - 73s 258ms/step - loss: 0.0154 - mse: 0.0154 - mae: 0.0914 - val_loss: 0.0253 - val_mse: 0.0253 - val_mae: 0.1114
508 4
282/282 [==============

282/282 [==============================] - 73s 258ms/step - loss: 0.0138 - mse: 0.0138 - mae: 0.0873 - val_loss: 0.0262 - val_mse: 0.0262 - val_mae: 0.1094
603 22
282/282 [==============================] - 73s 258ms/step - loss: 0.0143 - mse: 0.0143 - mae: 0.0896 - val_loss: 0.0271 - val_mse: 0.0271 - val_mae: 0.1134
604 29
282/282 [==============================] - 73s 258ms/step - loss: 0.0139 - mse: 0.0139 - mae: 0.0881 - val_loss: 0.0261 - val_mse: 0.0261 - val_mae: 0.1127
605 8
282/282 [==============================] - 73s 258ms/step - loss: 0.0144 - mse: 0.0144 - mae: 0.0894 - val_loss: 0.0247 - val_mse: 0.0247 - val_mae: 0.1071
606 38
282/282 [==============================] - 73s 258ms/step - loss: 0.0141 - mse: 0.0141 - mae: 0.0879 - val_loss: 0.0236 - val_mse: 0.0236 - val_mae: 0.1060
607 6
282/282 [==============================] - 73s 258ms/step - loss: 0.0147 - mse: 0.0147 - mae: 0.0893 - val_loss: 0.0253 - val_mse: 0.0253 - val_mae: 0.1103
608 23
282/282 [===============

282/282 [==============================] - 73s 258ms/step - loss: 0.0124 - mse: 0.0124 - mae: 0.0831 - val_loss: 0.0238 - val_mse: 0.0238 - val_mae: 0.1065
703 38
282/282 [==============================] - 73s 258ms/step - loss: 0.0137 - mse: 0.0137 - mae: 0.0871 - val_loss: 0.0261 - val_mse: 0.0261 - val_mae: 0.1125
704 39
282/282 [==============================] - 73s 258ms/step - loss: 0.0135 - mse: 0.0135 - mae: 0.0863 - val_loss: 0.0285 - val_mse: 0.0285 - val_mae: 0.1158
705 24
282/282 [==============================] - 73s 258ms/step - loss: 0.0133 - mse: 0.0133 - mae: 0.0864 - val_loss: 0.0248 - val_mse: 0.0248 - val_mae: 0.1084
706 0
282/282 [==============================] - 73s 258ms/step - loss: 0.0134 - mse: 0.0134 - mae: 0.0864 - val_loss: 0.0246 - val_mse: 0.0246 - val_mae: 0.1085
707 33
282/282 [==============================] - 73s 258ms/step - loss: 0.0133 - mse: 0.0133 - mae: 0.0865 - val_loss: 0.0247 - val_mse: 0.0247 - val_mae: 0.1103
708 1
282/282 [===============

282/282 [==============================] - 73s 258ms/step - loss: 0.0124 - mse: 0.0124 - mae: 0.0835 - val_loss: 0.0262 - val_mse: 0.0262 - val_mae: 0.1114
803 23
282/282 [==============================] - 73s 259ms/step - loss: 0.0129 - mse: 0.0129 - mae: 0.0846 - val_loss: 0.0252 - val_mse: 0.0252 - val_mae: 0.1111
804 38
282/282 [==============================] - 73s 258ms/step - loss: 0.0124 - mse: 0.0124 - mae: 0.0831 - val_loss: 0.0242 - val_mse: 0.0242 - val_mae: 0.1087
805 7
282/282 [==============================] - 73s 258ms/step - loss: 0.0129 - mse: 0.0129 - mae: 0.0850 - val_loss: 0.0237 - val_mse: 0.0237 - val_mae: 0.1066
806 0
282/282 [==============================] - 73s 259ms/step - loss: 0.0128 - mse: 0.0128 - mae: 0.0844 - val_loss: 0.0258 - val_mse: 0.0258 - val_mae: 0.1121
807 10
282/282 [==============================] - 73s 259ms/step - loss: 0.0126 - mse: 0.0126 - mae: 0.0836 - val_loss: 0.0236 - val_mse: 0.0236 - val_mae: 0.1077
808 9
282/282 [================

282/282 [==============================] - 73s 257ms/step - loss: 0.0120 - mse: 0.0120 - mae: 0.0822 - val_loss: 0.0261 - val_mse: 0.0261 - val_mae: 0.1144
903 48
282/282 [==============================] - 73s 258ms/step - loss: 0.0117 - mse: 0.0117 - mae: 0.0808 - val_loss: 0.0270 - val_mse: 0.0270 - val_mae: 0.1126
904 0
282/282 [==============================] - 73s 258ms/step - loss: 0.0131 - mse: 0.0131 - mae: 0.0854 - val_loss: 0.0271 - val_mse: 0.0271 - val_mae: 0.1126
905 2
282/282 [==============================] - 73s 258ms/step - loss: 0.0119 - mse: 0.0119 - mae: 0.0818 - val_loss: 0.0255 - val_mse: 0.0255 - val_mae: 0.1070
906 34
282/282 [==============================] - 73s 258ms/step - loss: 0.0115 - mse: 0.0115 - mae: 0.0811 - val_loss: 0.0256 - val_mse: 0.0256 - val_mae: 0.1090
907 15
282/282 [==============================] - 73s 258ms/step - loss: 0.0121 - mse: 0.0121 - mae: 0.0824 - val_loss: 0.0261 - val_mse: 0.0261 - val_mae: 0.1105
908 33
282/282 [===============